In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import numpy as np
# Batch size during training
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import warnings
warnings.filterwarnings("ignore")




def asmape(y_true, y_pred, mask=None):
    if mask is not None:
         y_true, y_pred = y_true[mask==1], y_pred[mask==1]
    if type(y_true) is list or type(y_pred) is list:
         y_true, y_pred = np.array(y_true), np.array(y_pred)
    len_ = len(y_true)
    tmp = 100 * (np.nansum(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))/len_)

    return tmp


class LoaderDataset(Dataset):
    def __init__(self, root_zebra, root_horse, root_masks, chanels=3):
        self.root_zebra = root_zebra
        self.root_horse = root_horse
        self.root_index = root_masks
        
        self.zebra_images = sorted(os.listdir(root_zebra))
        self.horse_images = sorted(os.listdir(root_horse))
        self.index = sorted(os.listdir(root_masks))

        self.length_dataset = max(len(self.zebra_images), len(self.horse_images))
        self.zebra_len = len(self.zebra_images)
        self.horse_len = len(self.horse_images)
        self.index_len = len(self.index)
        self.chanels = chanels

    def __len__(self):
        return self.length_dataset

    @staticmethod
    def custom_normalize(image):
        image = torch.tensor(image, dtype=torch.float32)
        min_val = torch.min(image)
        max_val = torch.max(image)
        scale = torch.clamp(max_val - min_val, min=1e-5)  # Evita divisão por zero
        image_normalized = 2 * (image - min_val) / scale - 1  # Escala para [-1, 1]
        return image_normalized, min_val, max_val

    def __getitem__(self, index):
        zebra_img = self.zebra_images[index % self.zebra_len]
        horse_img = self.horse_images[index % self.horse_len]
        index_ids = self.index[index % self.index_len]

        zebra_path = os.path.join(self.root_zebra, zebra_img)
        horse_path = os.path.join(self.root_horse, horse_img)
        index_path = os.path.join(self.root_index, index_ids)
        # print(zebra_path, horse_path, index_path)

        zebra_img = np.load(zebra_path)
        horse_img = np.load(horse_path)
        mask = np.load(index_path)

        if len(zebra_img.shape) > 3:
            zebra_img = zebra_img.reshape(32, 32, 3)
            horse_img = horse_img.reshape(32, 32, 3)

        zebra_img = np.transpose(zebra_img, (2, 0, 1))
        horse_img = np.transpose(horse_img, (2, 0, 1))

        if self.chanels == 2:
            zebra_img = zebra_img[:2, :, :]
            horse_img = horse_img[:2, :, :]
        elif self.chanels == 1:
            zebra_img = np.sum(zebra_img, axis=0, keepdims=True)
            horse_img = np.sum(horse_img, axis=0, keepdims=True)

        zebra_img, min_val_z, max_val_z = LoaderDataset.custom_normalize(zebra_img)
        horse_img, _, _ = LoaderDataset.custom_normalize(horse_img)

        mask = torch.tensor(mask, dtype=torch.float32)

        return zebra_img, horse_img.flatten(), min_val_z, max_val_z, mask
    



In [26]:

class Generator(nn.Module):
    def __init__(self, nz=3072, ngf=32, nc=3):
        super(Generator, self).__init__()
        self.nz = nz
        self.ngf = ngf
        self.nc = nc

        self.main = nn.Sequential(
            # Camada Linear para mapear o vetor de entrada para um tamanho de 8x8xngf*8
            nn.Linear(self.nz, self.ngf * 8 * 8 * 8),
            nn.ReLU(True),

            # Redimensiona para (ngf*8) x 8 x 8
            nn.Unflatten(1, (self.ngf * 8, 8, 8)),

            # Convoluções transpostas para aumentar gradualmente a resolução
            nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(self.ngf * 2, self.ngf, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),         
        
            # Ajuste final: reduzir a última camada para não aumentar o tamanho da imagem
            nn.ConvTranspose2d(self.ngf, self.nc, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Tanh()  # Normaliza a saída entre [-1, 1]
        )

    def forward(self, input):
        return self.main(input)


In [27]:


class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(Discriminator, self).__init__()
    
        self.nc = nc
        self.ndf = ndf

        self.main = nn.Sequential(
            # Camada 1: Convolução com stride 2 reduz a imagem para (image_size/2) x (image_size/2)
            nn.Conv2d(self.nc, self.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Camada 2: Convolução com stride 2 reduz a imagem para (image_size/4) x (image_size/4)
            nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Camada 3: Convolução com stride 2 reduz a imagem para (image_size/8) x (image_size/8)
            nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Camada 4: Convolução com stride 2 reduz a imagem para (image_size/16) x (image_size/16)
            nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Camada final: Convolução com kernel de (2, 2) para reduzir a imagem para 1 x 1
            nn.Conv2d(self.ndf * 8, 1, 2, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:




def asmape(y_true, y_pred, mask=None):
    if mask is not None:
         y_true, y_pred = y_true[mask==1], y_pred[mask==1]
    if type(y_true) is list or type(y_pred) is list:
         y_true, y_pred = np.array(y_true), np.array(y_pred)
    len_ = len(y_true)
    tmp = 100 * (np.nansum(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))/len_)

    return tmp



def add_masked_gaussian_noise(x: torch.Tensor, mask: torch.Tensor, sigma: float) -> torch.Tensor:
    # x: [B,C,H,W], mask: [B,1,H,W]
    inv = (1.0 - mask).expand(-1, x.size(1), -1, -1)
    noise = torch.randn_like(x) * sigma
    return x + inv * noise


def james_stein_reduce(losses: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Applies the James-Stein estimator to the mean of a vector of individual losses.

    Args:
        losses (torch.Tensor): 1D tensor of individual losses.
        eps (float): Numerical stability term.

    Returns:
        torch.Tensor: Smoothed mean of losses using James-Stein.
    """
    n = losses.numel()
    mu = losses.mean()
    S2 = torch.var(losses, unbiased=False)
    norm_sq = torch.sum((losses - mu)**2)
    shrinkage = torch.clamp(1 - ((n-2) * S2 / (norm_sq + eps)), min=0.0)
    # Estimator of the James-Stein
    js_mean = mu + shrinkage * (losses - mu).mean()
    return js_mean

def add_masked_gaussian_noise_vector(x: torch.Tensor, mask: torch.Tensor, sigma: float = 0.1) -> torch.Tensor:
    """
    Applies masked Gaussian noise to vector x, where the mask must have the same vector dimension.
    The original mask (usually [B, 1, H, W]) must be flattened and projected to the same shape as x.
    """
    # x: [B, F]
    # mask: [B, 1, H, W] → needs to be transformed to [B, F]
    B, F = x.size()

    # Achatar máscara para [B, H*W]
    mask_flat = mask.view(mask.size(0), -1)  # [B, H*W]

    # Design for the same number of features as x
    if mask_flat.size(1) != F:
        # Interpolate or repeat to adjust (depending on the case)
        mask_flat = torch.nn.functional.interpolate(mask_flat.unsqueeze(1), size=F, mode="nearest").squeeze(1)

    # cresats noise
    noise = torch.randn_like(x) * sigma

    # Applies noise where the mask is 0
    inv = (1.0 - mask_flat)
    return x + inv * noise

def train_and_val(loader, val_loader, nc, num_epochs=50, patience=100, device=None):
    
    
    netG = Generator(nz=nc*1024, nc=nc).to(device)
    netD = Discriminator(nc=nc).to(device)

    lr = 1e-5
    beta1 = 0.5
    best_loss = np.inf
    patience_counter = 0

    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
    criterion = nn.BCELoss(reduction="none")

    real_label = 1.
    fake_label = 0.

    print("Starting Training Loop...")

    for epoch in range(num_epochs):
        initial_sigma = 0.1
        warmup_epochs = 100
        sigma = max(0.0, initial_sigma * (1 - epoch / warmup_epochs))

        for i, (real, noise, _, _, masks) in enumerate(loader):
            noise = noise.to(device)
            real_gpu = real.to(device)
            masks = masks.to(device).view(-1, 1, 32, 32).float()

            real_noisy = add_masked_gaussian_noise(real_gpu, masks, sigma)
            noise_noisy = add_masked_gaussian_noise_vector(noise, masks, sigma)

            real_masked = real_noisy * masks

            ### train Discriminador ###
            netD.zero_grad()
            b_size = real_gpu.size(0)

            # Real
            label_real = torch.full((b_size,), real_label, device=device)
            output_real = netD(real_masked).view(-1)
            errD_real = james_stein_reduce(criterion(output_real, label_real))

            # Fake
            fake = netG(noise_noisy).detach()
            fake_masked = fake * masks
            label_fake = torch.full((b_size,), fake_label, device=device)
            output_fake = netD(fake_masked).view(-1)
            errD_fake = james_stein_reduce(criterion(output_fake, label_fake))

            errD = errD_real + errD_fake
            errD.backward()
            optimizerD.step()

            ### train Generator ###
            netG.zero_grad()
            label_real = torch.full((b_size,), real_label, device=device)
            outG = netG(noise_noisy)
            outG_masked = outG * masks
            outputG = netD(outG_masked).view(-1)
            errG = james_stein_reduce(criterion(outputG, label_real))
            errG.backward()
            optimizerG.step()

            if i % 100 == 0:
                print(f"[{epoch}/{num_epochs}][{i}/{len(loader)}] "
                      f"Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")

        # Validation with James-Stein
        avg_loss_G = validate(val_loader, netD=netD, netG=netG, device=device, criterion=criterion)

        if avg_loss_G < best_loss:
            best_loss = avg_loss_G
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping na época {epoch}!")
            break

    return netG


def validate(loader, netG, netD, device, criterion):
    netG.eval()
    netD.eval()

    total_loss_G = 0.0
    total_loss_D = 0.0
    num_batches = 0

    with torch.no_grad():
        for real_cpu, noise, _, _, masks in loader:
            real_cpu = real_cpu.to(device)
            noise = noise.to(device)
            masks = masks.to(device).view(-1, 1, 32, 32).float()
            noise = add_masked_gaussian_noise_vector(noise, masks, sigma=0.1)
            real_cpu = add_masked_gaussian_noise(real_cpu, masks, sigma=0.1)

            real_masked = real_cpu * masks
            fake = netG(noise)
            fake_masked = fake * masks

            output_real = netD(real_masked).view(-1)
            output_fake = netD(fake_masked).view(-1)

            label_real = torch.full_like(output_real, 1.0, device=device)
            label_fake = torch.full_like(output_fake, 0.0, device=device)

            loss_real = criterion(output_real, label_real)
            loss_fake = criterion(output_fake, label_fake)
            loss_D = james_stein_reduce(loss_real) + james_stein_reduce(loss_fake)
            loss_G = james_stein_reduce(criterion(output_fake, label_real))

            total_loss_D += loss_D.item()
            total_loss_G += loss_G.item()
            num_batches += 1

    avg_loss_D = total_loss_D / num_batches
    avg_loss_G = total_loss_G / num_batches
    print(f'Validation Loss_D: {avg_loss_D:.4f} \tValidation Loss_G: {avg_loss_G:.4f}')

    netG.train()
    netD.train()
    return avg_loss_G



def discriminator_loss(real_output, fake_output):
    criterion = nn.BCEWithLogitsLoss()
    real_loss = criterion(real_output, torch.ones_like(real_output, device=real_output.device))
    fake_loss = criterion(fake_output, torch.zeros_like(fake_output, device=fake_output.device))
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    criterion = nn.BCEWithLogitsLoss()
    return criterion(fake_output, torch.ones_like(fake_output, device=fake_output.device))


def test( gen, test_loader, taxa, fold, chanells, device):  
    gen.eval()

  
    with torch.no_grad():
        df = pd.DataFrame([],columns=['mae','asmape','mape','rmse','scale'], index=test_loader.dataset.horse_images)
        for (zebra, horse,std_val,mean_val, masks), name in zip(test_loader,test_loader.dataset.horse_images):
        
            zebra = zebra.to(device)
            horse = horse.to(device)
           
            # Converter k e min_val para tensores, mas movê-los para GPU somente se necessário
            std_val = torch.tensor(std_val) if not isinstance(std_val, torch.Tensor) else std_val
            mean_val = torch.tensor(mean_val) if not isinstance(mean_val, torch.Tensor) else mean_val
            
            # Gerar fake_zebra usando o gerador
            fake_zebra = gen(horse)
         
            # Mover apenas as imagens para CPU antes de operações subsequentes
            zebra = zebra.cpu()
            fake_zebra = fake_zebra.cpu()
            
             #Voltar para escala original 
            zebra = (zebra)*std_val + mean_val
            fake_zebra = (fake_zebra)* std_val + mean_val

             # Somar sobre o canal e achatar as imagens
            zebra = torch.sum(zebra, dim=1).flatten()
            fake_zebra = torch.sum(fake_zebra, dim=1).flatten()

            # Calcular as métricas
            zebra_np = zebra*masks
            fake_zebra_np = fake_zebra*masks

				    # Calcular as métricas corretamente
            mae_value = round(mae(zebra_np, fake_zebra_np), 3)
            mape_value = round(mape(zebra_np, fake_zebra_np) * 100, 3)
            rmse_value = round(np.sqrt(mse(zebra_np, fake_zebra_np)), 3)
            smape_value = round(asmape(zebra_np, fake_zebra_np,masks), 3)
            # Adicionar os resultados ao DataFram
            df.loc[name] = [mae_value, smape_value, mape_value, rmse_value, np.max(zebra.numpy()) - np.min(zebra.numpy())]

				# Salva o DataFrame em um arquivo CSV
        directory =  "./resultados/resultados_dc"
        if not os.path.exists(directory):
          os.makedirs(directory)
        df.to_csv(rf'{directory}/result_{str(chanells)}c_{taxa}_{fold}.csv')



In [29]:
TRAIN_DIR = os.path.abspath("../dataset_final")  
VAL_DIR = os.path.abspath("../dataset_final")  
INDEX_TRAIN = os.path.abspath("../dataset_final")  
INDEX_VAL = os.path.abspath("../dataset_final")  
INDEX_TEST = os.path.abspath("../dataset_final")  

In [ ]:

BATCH_SIZE = 1000
NUM_EPOCHS =50000
 
           
def main(in_channels):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    for taxa in ['10','20','30','40']:
        for fold in ['1','2','3','4','5']:  
        
            dataset = LoaderDataset(
                  root_zebra=os.path.join( TRAIN_DIR, "label", str(taxa), "folds", f"fold{fold}", "train"),
                  root_horse=os.path.join( TRAIN_DIR, "input", str(taxa), "folds", f"fold{fold}", "train"),
									root_masks=os.path.join( INDEX_TRAIN, "input", str(taxa), "folds", f"fold{fold}", "index_train"),
									chanels=in_channels
							  )
                
            val_dataset = LoaderDataset(
                root_zebra=os.path.join( VAL_DIR, "label", str(taxa), "folds", f"fold{fold}", "val"),
								root_horse=os.path.join( VAL_DIR, "input", str(taxa), "folds", f"fold{fold}", "val"),
								root_masks=os.path.join( INDEX_VAL, "input", str(taxa), "folds", f"fold{fold}", "index_val"),
								chanels=in_channels
						  )

            test_dataset = LoaderDataset(
                  root_zebra=os.path.join( VAL_DIR, "label", str(taxa), "folds", f"fold{fold}", "test"),
									root_horse=os.path.join( VAL_DIR, "input", str(taxa), "folds", f"fold{fold}", "test"),
									root_masks=os.path.join(INDEX_TEST, "input", str(taxa), "folds", f"fold{fold}", "index"),
									chanels=in_channels
						)
            			
            val_loader = DataLoader(
                val_dataset,
                batch_size=BATCH_SIZE,
                shuffle=False,
                pin_memory=False)
            
            loader = DataLoader(
                dataset,
                batch_size=BATCH_SIZE,
                shuffle=True,
           
                pin_memory=False)
            
            test_loader = DataLoader(
                    test_dataset,
                    batch_size=1,
                    shuffle=False,
                    pin_memory=False )
            
            #Treino        
            generator = train_and_val(loader, val_loader, nc=in_channels, num_epochs=NUM_EPOCHS,device=device)
            
            save_dir = f"./models_saved/dcgan/{in_channels}/{taxa}/fold{fold}"
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
                
            # Salvar modelo
            model_path = os.path.join(save_dir, "generator.pth")
            torch.save(generator.state_dict(), model_path)
            print(f"Modelo salvo em: {model_path}")

            # Teste
            print('Iniciando o teste ....')
            test(generator,test_loader=test_loader,taxa=taxa,fold=fold,chanells=in_channels, device=device)

import time

# import torch.multiprocessing as mp

if __name__ == '__main__':
    start_time = time.time()
  # Necessário no Windows para compatibilidade
    for i in [1,2,3]:
        main(i)
    total_time = time.time() - start_time
    print(f'{total_time/3600} hours')


Starting Training Loop...
[0/1000][0/2] Loss_D: 1.4338 Loss_G: 0.6915
Validation Loss_D: 1.3879 	Validation Loss_G: 0.6521
[1/1000][0/2] Loss_D: 1.4283 Loss_G: 0.7010
Validation Loss_D: 1.3911 	Validation Loss_G: 0.6194
[2/1000][0/2] Loss_D: 1.4240 Loss_G: 0.7203
Validation Loss_D: 1.3957 	Validation Loss_G: 0.5966
[3/1000][0/2] Loss_D: 1.4229 Loss_G: 0.7262
Validation Loss_D: 1.4017 	Validation Loss_G: 0.5754
[4/1000][0/2] Loss_D: 1.4198 Loss_G: 0.7254
Validation Loss_D: 1.4050 	Validation Loss_G: 0.5665
[5/1000][0/2] Loss_D: 1.4113 Loss_G: 0.7235
Validation Loss_D: 1.4119 	Validation Loss_G: 0.5587
[6/1000][0/2] Loss_D: 1.4117 Loss_G: 0.7189
Validation Loss_D: 1.4119 	Validation Loss_G: 0.5704
[7/1000][0/2] Loss_D: 1.4081 Loss_G: 0.7182
Validation Loss_D: 1.4027 	Validation Loss_G: 0.5959
[8/1000][0/2] Loss_D: 1.4050 Loss_G: 0.7137
Validation Loss_D: 1.3851 	Validation Loss_G: 0.6317
[9/1000][0/2] Loss_D: 1.4006 Loss_G: 0.7161
Validation Loss_D: 1.3720 	Validation Loss_G: 0.6704
[10/